## 車の燃費を予測する。前処理、特徴量エンジニアリング、予測、評価を行う

## 1. 前処理

In [1]:
import pandas as pd

/tmp/ipykernel_6781/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ["MPG", "Cylinders", "Displacement", "Horsepower", "Weight", "Acceleration", "Model_Year", "Origin"]
df = pd.read_csv(url, names=column_names, na_values="?", comment="\t", sep=" ", skipinitialspace=True)

In [3]:
df = df.dropna()
df = df.reset_index(drop=True)

In [4]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=0.8, random_state=1)
train_stats = df_train.describe().transpose()
numeric_column_names = ["Cylinders", "Displacement", "Horsepower", "Weight", "Acceleration"]
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()
df_train_norm[numeric_column_names] = df_train_norm[numeric_column_names].astype(float)
df_test_norm[numeric_column_names] = df_test_norm[numeric_column_names].astype(float)
for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, "mean"]
    std = train_stats.loc[col_name, "std"]
    df_train_norm.loc[:, col_name] = (df_train_norm.loc[:, col_name] - mean) / std
    df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean) / std

df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model_Year,Origin
203,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3
255,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1
72,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1
235,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1
37,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1


In [5]:
# Model_Year は 4つのバケットに分割する
import torch

boundaries = torch.tensor([73, 76, 79])
v = torch.tensor(df_train_norm["Model_Year"].values)
df_train_norm["Model Year Bucketed"] = torch.bucketize(v, boundaries, right=True)
v = torch.tensor(df_test_norm["Model_Year"].values)
df_test_norm["Model Year Bucketed"] = torch.bucketize(v, boundaries, right=True)

numeric_column_names += ["Model Year Bucketed"]

In [6]:
# Origin(生産地域)は One-Hot エンコーティングする
from torch.nn.functional import one_hot

total_origin = len(set(df_train_norm["Origin"]))
origin_encoded = one_hot(torch.from_numpy(df_train_norm["Origin"].values) % total_origin)
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

origin_encoded = one_hot(torch.from_numpy(df_test_norm["Origin"].values) % total_origin)
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()

In [7]:
# 目的変数としてらえるテンソルを生成
y_train = torch.tensor(df_train_norm["MPG"].values).float()
y_test = torch.tensor(df_test_norm["MPG"].values).float()

## 2. モデルの訓練

In [8]:
from torch.utils.data import DataLoader, TensorDataset

train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [9]:
import torch.nn as nn

hidden_units = [8, 4]
input_size = x_train.shape[1]
all_layers = []
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 1))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [10]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [11]:
torch.manual_seed(1)
num_epochs = 200
log_epochs = 20
for epoch in range(num_epochs):
    loss_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:, 0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist_train += loss.item()

    if epoch % log_epochs == 0:
        print(f"{epoch=} Loss: {loss_hist_train/len(train_dl):.4f}")

epoch=0 Loss: 536.1047
epoch=20 Loss: 8.4361
epoch=40 Loss: 7.8695
epoch=60 Loss: 7.1891
epoch=80 Loss: 6.7062
epoch=100 Loss: 6.7599
epoch=120 Loss: 6.3124
epoch=140 Loss: 6.6864
epoch=160 Loss: 6.7648
epoch=180 Loss: 6.2156


In [ ]:
with torch.no_grad():
    pred = model(x_test.float())[:, 0]
    loss = loss_fn(pred, y_test)